In [3]:
import os
import json

In [4]:
def get_name_and_extension(file_name):
    """
    get name and extension of a file_name
    :param file_name:
    :return:
    """
    tokens = file_name.split('.')
    return {'name': '.'.join(tokens[0:-1]), 'ext': tokens[-1]}

In [34]:
def convert_to_dict(data_file):
    dict_data = "["
    while True:
        line = data_file.readline()

        if dict_data != '[' and line!= "":
            dict_data += ','

        if line != "":
            dict_data += line
        else:
            break

    dict_data += ']'

    # print(dict_data)

    return dict_data

In [35]:
def process_file_labels(file):
    diff_labels = {}
    for label in file:
        title = label['title']
        label['match'] = label['match'].split('_')[0]
        main_label = title.split('-')[0]
        if main_label in diff_labels:
            diff_labels[main_label].append(label)
        else:
            diff_labels[main_label] = [label]
    
    return diff_labels

In [28]:
def process_file_same_labels(file):
    diff_labels = {'all': []}
    for label in file:
        title = label['title']
        label['match'] = label['match'].split('_')[0]
        label['title'] = title.split('-')[0]
        diff_labels['all'].append(label)
    
    return diff_labels

In [36]:
def generate_separate_label_files(original_folder, redone_folder):
    for subdir, dirs, files in os.walk(redone_folder):
        for filename in files:
            name_extension = get_name_and_extension(filename)
            name = name_extension['name']
            ext = name_extension['ext'].lower()

            if ext == 'json':
                with open(redone_folder+filename, 'r') as redone_file:
                    dict_data = convert_to_dict(redone_file)
                    redone_label = json.loads(dict_data)
                    redone_parsed_label = process_file_labels(redone_label)
#                     redone_parsed_label = process_file_same_labels(redone_label)
    
    for subdir, dirs, files in os.walk(original_folder):
        for filename in files:
            name_extension = get_name_and_extension(filename)
            name = name_extension['name']
            ext = name_extension['ext'].lower()

            if ext == 'json':
                with open(original_folder+filename, 'r') as original_file:
                    dict_data = convert_to_dict(original_file)               
                    original_label = json.loads(dict_data)
                    original_parsed_label = process_file_labels(original_label)
#                     original_parsed_label = process_file_same_labels(original_label)
                
    
    for key in redone_parsed_label:
        red_file = []
        for label in redone_parsed_label[key]:
            red_file.append(label)
            
        with open(redone_folder+key+'.json', 'w') as outfile:
            json.dump(red_file, outfile)
        
        
    for key in original_parsed_label:
        org_file = []
        for label in original_parsed_label[key]:
            org_file.append(label)
            
        with open(original_folder+key+'.json', 'w') as outfile:
            json.dump(org_file, outfile)
    

In [37]:
org = 'data/dota2Seq/raw/original/'
red = 'data/dota2Seq/raw/redone/'
generate_separate_label_files(org, red)